### Set Environment

In [24]:
import os
import json
import warnings
import pandas as pd
from glob2 import glob
warnings.filterwarnings("ignore")


### List all Output paths

In [25]:
jsonFiles = \
    glob('/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/*/posts/*.json'
         )  # Can be used absolute or relative paths


#### Create Aggregate folder

In [26]:
os.makedirs("/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data")

### Function to
* __Merge all data in files__
* __Normalizes merged .json into dataframe__
* __Saves as csv__

In [12]:
tiktok_data = pd.DataFrame([])


def merge_JsonFiles(filename):
    for f1 in filename:
        global tiktok_data
        with open(f1, "r") as infile:
            data = json.load(infile)
        tiktok_data = tiktok_data.append(pd.json_normalize(data), ignore_index=True)
        tiktok_data.to_csv(
            "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/tiktok.csv",
            index=False,
        )

merge_JsonFiles(jsonFiles)

### Cleaning
#### Delete duplicate data: by 'webVideoUrl'

In [5]:
# Delete duplicate rows based on specific columns

tiktok_data = tiktok_data.drop_duplicates(subset=['webVideoUrl'],
                                          keep=False)


### Data Summaries
#### To Flourish:

### Visuals
#### 1. User from 'webVideoUrl'

In [4]:
source = pd.DataFrame([])
source["source"] = tiktok_data['webVideoUrl'].str.split('/').str[3]
source["source"].value_counts().rename_axis("unique_sources").reset_index(
    name="counts"
).to_csv("/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/source.csv")

#### 2. Virial Video: 'webvideoUrl' 

In [5]:
tiktok_data["videoUrl"].value_counts(ascending=True).rename_axis(
    "unique_videos"
).reset_index(name="counts").to_csv(
    "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/videos.csv"
)


#### 3. Network Analysis data
 * Source = 'authorMeta.name'
 * Target = 👇

In [16]:
pd.DataFrame(tiktok_data.mentions.values.tolist()).add_prefix('target_').to_csv(
    "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/target.csv"
)

#### Hashtags

In [23]:
pd.Series(
    [y.get("name") for x in tiktok_data["hashtags"] for y in x]
).value_counts().rename_axis("hahtag").reset_index(name="freq").to_csv(
    "/Users/ivanmusebe/Desktop/tiktok-hashtag-analysis/data/aggregates_data/hashtags.csv"
)

#### 3. Sentimental Analysis by 'text'